In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [38]:
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)

# Fit the model
my_model_1.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:39:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [244]:
# Tunned XGBoostRegressor

from xgboost import XGBRegressor

my_model_1 = XGBRegressor(silent=True, 
                              scale_pos_weight=1,
                              learning_rate=0.045,
                              colsample_bylevel=1,
                              colsample_bynode=1,
                              colsample_bytree = 0.44,
                              subsample = 0.78,
                              n_estimators=1000, 
                              reg_alpha = 0.1,
                              max_depth=5, 
                              gamma=0,
                              n_jobs=4,
                              reg_lambda=1.01,
                              min_child_weight=1)

# Fit the model
my_model_1.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.44, gamma=0,
             importance_type='gain', learning_rate=0.045, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.1, reg_lambda=1.01, scale_pos_weight=1, seed=None,
             silent=True, subsample=0.78, verbosity=1)

In [245]:
from sklearn.metrics import mean_absolute_error

# Get predictions
predictions_1 = my_model_1.predict(X_valid)

# Calculate MAE
mae_1 = mean_absolute_error(y_valid, predictions_1)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 15724.78496896404


In [45]:
def get_scores(model, X_train, X_valid, y_train, y_valid):
    # Fit the model
    model.fit(X_train, y_train, 
                 early_stopping_rounds=5, 
                 eval_set=[(X_valid, y_valid)], 
                 verbose=False)
    # Get predictions
    preds_val = model.predict(X_valid)
    # Calculate MAE
    mae = mean_absolute_error(y_valid, preds_val)
    return mae

In [54]:
for lr in range(1, 20):
    lr = lr * 0.01
    print(lr)
#     model = XGBRegressor(n_estimators=1000, learning_rate=lr, n_jobs=4)
    model = XGBRegressor(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=lr,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)
    scores = get_scores(model, X_train, X_valid, y_train, y_valid)
    print('Learning rate is {}. Mae is {}'.format(str(lr), str(scores)))

0.01
[14:49:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Learning rate is 0.01. Mae is 16249.789276541096
0.02
[14:49:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Learning rate is 0.02. Mae is 16293.870130565068
0.03
[14:49:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Learning rate is 0.03. Mae is 16528.84249250856
0.04
[14:49:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

Learning rate is 0.17. Mae is 17348.95943921233
0.18
[14:49:49] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Learning rate is 0.18. Mae is 17626.245465004282
0.19
[14:49:49] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Learning rate is 0.19. Mae is 17771.04197880993


In [55]:
for ne in range(1, 12):
    n_estimators = ne * 100
#     model = XGBRegressor(n_estimators=n_estimators, learning_rate=0.05, n_jobs=4)
    model = XGBRegressor(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.05,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      n_estimators=n_estimators, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)
    scores = get_scores(model, X_train, X_valid, y_train, y_valid)
    print('n_estimators is {}. Mae is {}'.format(str(n_estimators), str(scores)))

d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:51:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
n_estimators is 100. Mae is 16871.14383561644
[14:51:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


n_estimators is 200. Mae is 15992.658229880137
[14:51:52] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

n_estimators is 900. Mae is 15953.125093642979
[14:52:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


n_estimators is 1000. Mae is 15953.125093642979
[14:52:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


n_estimators is 1100. Mae is 15953.125093642979


In [175]:
for reg_lambda in range(95, 110):
    reg_lambda = reg_lambda * 0.01
    print(reg_lambda)
#     model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
    model = XGBRegressor(silent=True, 
                      scale_pos_weight=1,
                      learning_rate=0.045,  
                      colsample_bytree = 0.44,
                      subsample = 0.78,
                      n_estimators=1000, 
                      reg_alpha = 0.1,
                      max_depth=5, 
                      gamma=10,
                      n_jobs=4,
                      reg_lambda=1.01)

    scores = get_scores(model, X_train, X_valid, y_train, y_valid)
    print('reg_lambda is {}. Mae is {}'.format(str(reg_lambda), str(scores)))

0.9500000000000001


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


reg_lambda is 0.9500000000000001. Mae is 16004.789102632705
0.96


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


reg_lambda is 0.96. Mae is 16097.077094927226
0.97


...

reg_lambda is 1.07. Mae is 15766.526407320205
1.08


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


reg_lambda is 1.08. Mae is 15995.528333690068
1.09


d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


reg_lambda is 1.09. Mae is 15929.40598244863


In [246]:
# Tunned XGBoostRegressor

from xgboost import XGBRegressor

my_model_1 = XGBRegressor(silent=True, 
                              scale_pos_weight=1,
                              learning_rate=0.045,
                              colsample_bylevel=1,
                              colsample_bynode=1,
                              colsample_bytree = 0.44,
                              subsample = 0.78,
                              n_estimators=1000, 
                              reg_alpha = 0.1,
                              max_depth=5, 
                              gamma=0,
                              n_jobs=4,
                              reg_lambda=1.01,
                              min_child_weight=1)

# Fit the model
my_model_1.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

d:\kagglelearn\venv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.44, gamma=0,
             importance_type='gain', learning_rate=0.045, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.1, reg_lambda=1.01, scale_pos_weight=1, seed=None,
             silent=True, subsample=0.78, verbosity=1)

In [247]:
preds_test = model.predict(X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('../output/submission.csv', index=False)